## Image Classification from Kera Models

## 1. Overview of Fruit Data Set <a name="introduction"></a>

We will use the Fruits-360 data from https://github.com/Horea94/Fruit-Images-Dataset to demonstrate how to apply kera CNN models for image classification tasks. The data set contains 120 classes of fruits and vegetables. There are 61488 images in the training data set, and 20622 images in the test data set. The image size is 100*100. 
- Reference: "Horea Muresan, Mihai Oltean, Fruit recognition from images using deep learning, Acta Univ. Sapientiae, Informatica Vol. 10, Issue 1, pp. 26-42, 2018."


#### Load packages

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.preprocessing import image
from keras.models import Model
#from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
#from keras.applications.resnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


## Preprocess images and prepare data using keras.flow_from_directory  

In [2]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies
dir_training = '/cas/DeepLearn/data/Fruit-Images-Dataset/Training'
train_generator = train_datagen.flow_from_directory(
    directory=dir_training,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=128,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 61488 images belonging to 120 classes.


In [3]:
## check class label and its index
print("Check class label and its index from ImageDataGenerator:")
train_generator.class_indices

Check class label and its index from ImageDataGenerator:


{'Apple Braeburn': 0,
 'Apple Crimson Snow': 1,
 'Apple Golden 1': 2,
 'Apple Golden 2': 3,
 'Apple Golden 3': 4,
 'Apple Granny Smith': 5,
 'Apple Pink Lady': 6,
 'Apple Red 1': 7,
 'Apple Red 2': 8,
 'Apple Red 3': 9,
 'Apple Red Delicious': 10,
 'Apple Red Yellow 1': 11,
 'Apple Red Yellow 2': 12,
 'Apricot': 13,
 'Avocado': 14,
 'Avocado ripe': 15,
 'Banana': 16,
 'Banana Lady Finger': 17,
 'Banana Red': 18,
 'Beetroot': 19,
 'Blueberry': 20,
 'Cactus fruit': 21,
 'Cantaloupe 1': 22,
 'Cantaloupe 2': 23,
 'Carambula': 24,
 'Cauliflower': 25,
 'Cherry 1': 26,
 'Cherry 2': 27,
 'Cherry Rainier': 28,
 'Cherry Wax Black': 29,
 'Cherry Wax Red': 30,
 'Cherry Wax Yellow': 31,
 'Chestnut': 32,
 'Clementine': 33,
 'Cocos': 34,
 'Dates': 35,
 'Eggplant': 36,
 'Ginger Root': 37,
 'Granadilla': 38,
 'Grape Blue': 39,
 'Grape Pink': 40,
 'Grape White': 41,
 'Grape White 2': 42,
 'Grape White 3': 43,
 'Grape White 4': 44,
 'Grapefruit Pink': 45,
 'Grapefruit White': 46,
 'Guava': 47,
 'Hazelnut

In [4]:
dir_test = '/cas/DeepLearn/data/Fruit-Images-Dataset/Test'
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies
test_generator = test_datagen.flow_from_directory(
    directory=dir_test,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=128,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

Found 20622 images belonging to 120 classes.


## Fine-tune existing models for your own data sets

In [5]:
## remove top layer
input_tensor = keras.Input(shape=(224, 224, 3))
## load model with pretrained weights
## The pretrained weights is not necessary when all the parameters are trainable
resnet50_base = keras.applications.resnet.ResNet50(include_top=False,  weights='imagenet', input_tensor=input_tensor, pooling='avg')


In [6]:
resnet50_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

## Add two dense layers on the top for classification

In [7]:
x = resnet50_base.output
#x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
# and a final classification layer with 120 classes
n_classes = 120
prediction = keras.layers.Dense(n_classes, activation='softmax')(x) #final layer with softmax activation
## the final model for your own data
resnet50_fruit=keras.Model(inputs=resnet50_base.input,outputs=prediction)

## Print out model summary

In [8]:
resnet50_fruit.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [9]:
#resnet50_fruit.get_weights()

In [10]:
## fine-tune specified layers
'''
n_layers = len(resnet50_fruit.layers)
for i, layer in enumerate(resnet50_fruit.layers):
    if i<(n_layers-2):
        layer.trainable=False
    else:
        layer.trainable=True
'''


'\nn_layers = len(resnet50_fruit.layers)\nfor i, layer in enumerate(resnet50_fruit.layers):\n    if i<(n_layers-2):\n        layer.trainable=False\n    else:\n        layer.trainable=True\n'

## Train on single GPU

In [11]:
model = resnet50_fruit
adamOpt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=adamOpt,loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy
# avoid OOM with smaller batch size
train_generator.batch_size = 64 ## smaller batch size for single GPU 
step_size_train=train_generator.n//train_generator.batch_size
n_epochs = 5
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=n_epochs,
                   #validation_data=test_generator
                   )

Epoch 1/5
960/960 [==============================] - 449s 468ms/step - loss: 0.1716 - accuracy: 0.9558
Epoch 2/5
960/960 [==============================] - 437s 456ms/step - loss: 0.0365 - accuracy: 0.9907
Epoch 3/5
960/960 [==============================] - 437s 455ms/step - loss: 0.0285 - accuracy: 0.9928
Epoch 4/5
960/960 [==============================] - 438s 456ms/step - loss: 0.0262 - accuracy: 0.9927
Epoch 5/5
960/960 [==============================] - 439s 458ms/step - loss: 0.0216 - accuracy: 0.9940


## Evaluate on test data set using evaluate_generator()

In [12]:
print("Evaluation metric:",model.metrics_names)
model.evaluate_generator(generator=test_generator)        

Evaluation metric: ['loss', 'accuracy']


[2.0997447791160084e-05, 0.9847735166549683]

##  Train model on multiple GPU using multi_gpu_model

In [20]:
## remove top layer
input_tensor = keras.Input(shape=(224, 224, 3))
## load model with pretrained weights
## The pretrained weights is not necessary when all the parameters are trainable
resnet50_base_gpus = keras.applications.resnet.ResNet50(include_top=False,  weights='imagenet', input_tensor=input_tensor, pooling='avg')

x = resnet50_base_gpus.output
#x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
# and a final classification layer with 120 classes
n_classes = 120
prediction = keras.layers.Dense(n_classes, activation='softmax')(x) #final layer with softmax activation
## the final model for your own data
resnet50_fruit_gpus=keras.Model(inputs=resnet50_base_gpus.input,outputs=prediction)

In [21]:
resnet50_fruit_gpus.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [22]:
from keras.utils import multi_gpu_model
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback
## hack due to mismatch verison of tf and keras
print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

tf.__version__ is 2.1.0
tf.keras.__version__ is: 2.2.4-tf


In [17]:
#import gc
#print(gc.collect())
# keras.clear_session()

## Turn off easger mode to use multiple GPU training

In [23]:
tf.compat.v1.disable_eager_execution() ## turn off eager model to use multiple GPU

## compile and start training with multiple GPU
model = resnet50_fruit_gpus
parallel_model  = multi_gpu_model(model,gpus=2)
adamOpt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
parallel_model.compile(optimizer=adamOpt,loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy
n_epochs = 5
train_generator.batch_size = 128 ## larger batch size for multiple GPUs 


Epoch 1/1
480/480 [==============================] - 1249s 3s/step - loss: 0.1563 - accuracy: 0.9604


In [26]:
step_size_train=train_generator.n//train_generator.batch_size
parallel_model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=n_epochs
                   #validation_data=test_generator
                   )

Epoch 1/5
480/480 [==============================] - 1234s 3s/step - loss: 0.0249 - accuracy: 0.9936
Epoch 2/5
480/480 [==============================] - 1236s 3s/step - loss: 0.0327 - accuracy: 0.9919
Epoch 3/5
480/480 [==============================] - 1238s 3s/step - loss: 0.0079 - accuracy: 0.9981
Epoch 4/5
480/480 [==============================] - 1240s 3s/step - loss: 5.8236e-05 - accuracy: 1.0000
Epoch 5/5
480/480 [==============================] - 1233s 3s/step - loss: 2.2919e-05 - accuracy: 1.0000


## Evaluate on test data set using evaluate_generator()

In [27]:
print("Evaluation metric:",parallel_model.metrics_names)
parallel_model.evaluate_generator(generator=test_generator)

Evaluation metric: ['loss', 'accuracy']


[1.27724248955019e-07, 0.997284471988678]

In [28]:
parallel_model.evaluate_generator(generator=train_generator)

[2.1271821424306836e-06, 1.0]